In [204]:
import billboard
import webbrowser
from itertools import zip_longest
from collections import defaultdict
import base64
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import spotipy
import spotipy.util
import sys
import requests, contextlib, re, os
import time
import json
import datetime
import math

In [2]:
# #create urls:
# years = list(range(1941, 2017))
# year_dict = {}
# for year in years:
#     year_dict[year] = ('http://billboardtop100of.com/' + str(year) + '-2/')
# print(year_dict)


In [3]:
# #retrieve the tables
# all_data = []
# error_urls = []

# for year, url in year_dict.items():
# #     print('processing ' + url)
#     try:
#         df = pd.read_html(url)[0]
#         df[3] = year
#         all_data.append(df)
#     except:
# #         print('error' + url)
#         error_urls.append(url)
#         continue
# print(error_urls)

In [4]:
#create the dataframe
# df = pd.concat(all_data)
# df.columns=['Rank' , 'Artist', 'Song', 'Year']
# df.to_csv('billboard_dirty_output.csv')
# df = pd.read_csv('billboard_cleaned.csv', usecols=['Rank' , 'Artist', 'Song', 'Year'] , encoding='latin-1')
# df.head()
# df2 = pd.read_csv("data.csv")
# incomp_df = pd.read_csv('incomp_artist_id.csv', usecols=['Rank' , 'Artist', 'Song', 'Year', 'Artist_ID'] , encoding='latin-1')

In [205]:
# client id: client seceret
app_key = base64.b64encode(b"a1bf020b945647f5806538ca4c35754c:e747cd9707f9488f99653515a1c02929").decode('utf-8')
r = requests.post("https://accounts.spotify.com/api/token", data={'grant_type': 'client_credentials'}, headers={'Authorization': 'Basic ' + str(app_key)})
access_token = r.json()['access_token']
print(access_token)

BQAByHxKLuA3-g0GawtEUZoc-p5ZL_3ziiaVy8Nts151JFAyLe3mKiIcSvjzDSU2pulBQsIkqUqfj_1W8uQ


In [6]:
artist_ids_df = pd.read_csv('artist_ids.csv', usecols=['Rank' , 'Artist', 'Song', 'Year', 'Artist_ID'] , encoding='latin-1')

In [37]:
#transform the artist into an artist_id
# throttle_time = 1.00 #seconds

# errored_artists = []

# def artist_id (artist):
#     request = requests.get('https://api.spotify.com/v1/search', headers={'Authorization': 'Bearer ' + access_token}, params={'q': artist, 'type':'artist', 'market': 'US'})
#     results = request.json()["artists"]["items"]
#     if len(results) == 0:
#         if ' and' in artist: 
#             artist_1 = artist.split(' and')[0]
#             artist_id(artist_1)
#             return artist_id(artist_1)
#         elif '&' in artist: 
#             artist_1 = artist.split('&')[0]
#             artist_id(artist_1)
#             return artist_id(artist_1)
#         elif ' And' in artist: 
#             artist_1 = artist.split(' And')[0]
#             artist_id(artist_1)
#             return artist_id(artist_1)
#         elif 'feat.' in artist:
#             artist_1 = artist.split('feat.')[0]
#             artist_id(artist_1)
#             return artist_id(artist_1)
#         elif 'Feat.' in artist:
#             artist_1 = artist.split('Feat.')[0]
#             artist_id(artist_1)
#             return artist_id(artist_1)
#         elif ',' in artist: 
#             artist_1 = artist.split(',')[0]
#             artist_id(artist_1)
#             return artist_id(artist_1)
#         elif ' With' in artist: 
#             artist_1 = artist.split(' With')[0]
#             artist_id(artist_1)
#             return artist_id(artist_1)
#         elif ' with' in artist: 
#             artist_1 = artist.split(' with')[0]
#             artist_id(artist_1)
#             return artist_id(artist_1)
#         else:
#             errored_artists.append(artist)
#             print('error w/ ' + artist)
#             return 'null'
#     else:
#         top_match = results[0]
#         spotify_id = top_match["id"]
#         return spotify_id
#         time.sleep(throttle_time)

# for index, row in artist_ids_df.iterrows():
#     cur_id = row['Artist_ID']
#     if not isinstance(cur_id, str) and math.isnan(cur_id):
#         try:
#             artist_ids_df.loc[index, 'Artist_ID'] = artist_id(row['Artist'])
#         except OSError:
#             print('ERROR')
# #     else: 
# #         print (row['Artist_ID'])
        
        
# # df['Artist'].apply(artist_id)

# # df['Artist_info'] = df['artist_id'].apply(artist_info)
# # artist_info_dict = artist_info(artist = 'Michael Jackson')
# # print(artist_info_dict)

In [35]:
# artist_ids_df.to_csv('artist_ids.csv')

In [43]:
df = pd.read_csv('artist_ids_clean.csv', usecols=['Rank' , 'Artist', 'Song', 'Year', 'Artist_ID'] , encoding='latin-1')
len(df)

6710

In [140]:
unique_artists = df['Artist_ID'].unique()

def get_genre_if_exists(genres):
    if (len(genres) > 0):
        return genres[0]
    return "unassigned"

def get_followers_if_exists(followers):
    if (len(followers) > 0):
        return followers.get("total", 0)
    return 0
    
def get_img_if_exists(images):
    if (len(images) > 0):
        return images[0]["url"]
    return "https://i.redd.it/6dg6m5e556m11.jpg"

def extract_artist_info(artist):
    artist_info = {}
    artist_info['genre'] = get_genre_if_exists(artist["genres"])
    artist_info['followers'] = get_followers_if_exists(artist["followers"])
    artist_info['image'] = get_img_if_exists(artist["images"])                                      
    return artist_info


def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z
    
all_ids_info = {} 
for i in range(0, len(unique_artists), 50):
    print ("Processing " + str(i) + " of " + str(len(unique_artists)))
    next_50 = min(i + 50, len(unique_artists))
    ids = unique_artists[i:next_50]
    comma_sep_ids = ",".join(ids)
    response = requests.get('https://api.spotify.com/v1/artists', headers={'Authorization': 'Bearer ' + access_token}, params={'ids': comma_sep_ids})
    artists = response.json()['artists']
    ids_to_artist_info = {artist["id"] : extract_artist_info(artist) for artist in artists}
    all_ids_info = merge_two_dicts(ids_to_artist_info, all_ids_info)

print (all_ids_info)

Processing 0 of 2589
Processing 50 of 2589
Processing 100 of 2589
Processing 150 of 2589
Processing 200 of 2589
Processing 250 of 2589
Processing 300 of 2589
Processing 350 of 2589
Processing 400 of 2589
Processing 450 of 2589
Processing 500 of 2589
Processing 550 of 2589
Processing 600 of 2589
Processing 650 of 2589
Processing 700 of 2589
Processing 750 of 2589
Processing 800 of 2589
Processing 850 of 2589
Processing 900 of 2589
Processing 950 of 2589
Processing 1000 of 2589
Processing 1050 of 2589
Processing 1100 of 2589
Processing 1150 of 2589
Processing 1200 of 2589
Processing 1250 of 2589
Processing 1300 of 2589
Processing 1350 of 2589
Processing 1400 of 2589
Processing 1450 of 2589
Processing 1500 of 2589
Processing 1550 of 2589
Processing 1600 of 2589
Processing 1650 of 2589
Processing 1700 of 2589
Processing 1750 of 2589
Processing 1800 of 2589
Processing 1850 of 2589
Processing 1900 of 2589
Processing 1950 of 2589
Processing 2000 of 2589
Processing 2050 of 2589
Processing 2100

In [158]:
df['Genre'] = ""
for index, row in df.iterrows():
    cur_id = row['Artist_ID']
    genre = all_ids_info[cur_id]['genre']
    print ("Adding: " + genre  + " to: " + row['Artist'])
    df.loc[index, 'Genre'] = genre

df['Followers'] = int()
for index, row in df.iterrows():
    cur_id = row['Artist_ID']
    followers = all_ids_info[cur_id]['followers']
    print ("Adding: " + str(followers) + " to: " + row['Artist'])
    df.loc[index, 'Followers'] = followers

df['Img'] = ""
for index, row in df.iterrows():
    cur_id = row['Artist_ID']
    image = all_ids_info[cur_id]['image']
    print ("Adding: " + image + " to: " + row['Artist'])
    df.loc[index, 'Img'] = image

Adding: adult standards to: Glenn Miller
Adding: unassigned to: Sammy Kaye
Adding: adult standards to: Artie Shaw
Adding: adult standards to: The Andrews Sisters
Adding: big band to: Jimmy Dorsey
Adding: big band to: Jimmy Dorsey
Adding: adult standards to: Glenn Miller
Adding: adult standards to: Duke Ellington
Adding: adult standards to: Billie Holiday
Adding: big band to: Jimmy Dorsey
Adding: unassigned to: Horace Heidt
Adding: unassigned to: Freddy Martin
Adding: adult standards to: The Ink Spots
Adding: big band to: Jimmy Dorsey
Adding: adult standards to: Glenn Miller
Adding: big band to: Jimmy Dorsey
Adding: adult standards to: Harry James
Adding: unassigned to: Tommy Dorsey & Frank Sinatra
Adding: adult standards to: Glenn Miller
Adding: adult standards to: Benny Goodman
Adding: country gospel to: The Sons of the Pioneers
Adding: christmas to: Xavier Cugat
Adding: big band to: Jimmy Dorsey
Adding: big band to: Guy Lombardo
Adding: adult standards to: Vera Lynn
Adding: unassigne

Adding: unassigned to: Kay Kyser
Adding: adult standards to: Dick Haymes and Helen Forrest
Adding: adult standards to: Andrews Sisters and Les Paul Trio
Adding: adult standards to: Perry Como
Adding: adult standards to: Jo Stafford
Adding: unassigned to: Modernaires and Paula Kelly
Adding: adult standards to: Bing Crosby
Adding: unassigned to: The King Cole Trio
Adding: big band to: Guy Lombardo
Adding: adult standards to: Vaughn Monroe
Adding: adult standards to: Bing Crosby
Adding: unassigned to: Dusty Fletcher
Adding: unassigned to: Eddy Howard
Adding: adult standards to: Peggy Lee
Adding: unassigned to: Kay Kyser
Adding: unassigned to: Francis Craig
Adding: unassigned to: The Harmonicats
Adding: big band to: Ted Weems
Adding: british dance band to: Ray Noble and Buddy Clark
Adding: country gospel to: Tex Williams
Adding: adult standards to: Vaughn Monroe
Adding: easy listening to: The Three Suns
Adding: big band to: Al Jolson
Adding: unassigned to: Larry Green
Adding: unassigned to

Adding: adult standards to: Perry Como
Adding: adult standards to: Guy Mitchell
Adding: adult standards to: Fats Domino
Adding: rockabilly to: Sanford Clark
Adding: deep adult standards to: Don Robertson
Adding: unassigned to: Bing Crosby and Grace Kelly
Adding: adult standards to: Little Richard
Adding: adult standards to: Teresa Brewer
Adding: song poem to: Nervous Norvus
Adding: adult standards to: Ames Brothers
Adding: adult standards to: Teresa Brewer
Adding: merseybeat to: Lonnie Donegan
Adding: unassigned to: Dream Weavers
Adding: adult standards to: Pat Boone
Adding: adult standards to: Frank Sinatra
Adding: unassigned to: Eddie Heywood
Adding: adult standards to: Andy Williams
Adding: adult standards to: Eddie Fisher
Adding: country gospel to: George Hamilton Iv
Adding: adult standards to: Eddie Fisher
Adding: adult standards to: Jerry Vale
Adding: ragtime to: Dick Hyman
Adding: deep adult standards to: Gale Storm
Adding: deep adult standards to: Joe Valino
Adding: unassigned 

Adding: adult standards to: Everly Brothers
Adding: american choir to: Mormon Tabernacle Choir
Adding: unassigned to: Ernie Fields Orchestra
Adding: unassigned to: Paul Evans and The Curls
Adding: adult standards to: Percy Faith
Adding: adult standards to: Jim Reeves
Adding: adult standards to: Everly Brothers
Adding: brill building pop to: Johnny Preston
Adding: unassigned to: Mark Dinning
Adding: adult standards to: Brenda Lee
Adding: christmas to: Elvis Presley
Adding: bmore to: Jimmy Jones
Adding: christmas to: Elvis Presley
Adding: adult standards to: Chubby Checker
Adding: adult standards to: Connie Francis
Adding: adult standards to: Bobby Rydell
Adding: adult standards to: Brothers Four
Adding: brill building pop to: Jack Scott
Adding: country gospel to: Marty Robbins
Adding: unassigned to: Hollywood Argyles
Adding: adult standards to: Connie Francis
Adding: adult standards to: Brenda Lee
Adding: adult standards to: Brian Hyland
Adding: adult standards to: Roy Orbison
Adding: a

Adding: unassigned to: Bill Pursell
Adding: adult standards to: Tony Bennett
Adding: brill building pop to: Miracles
Adding: classic girl group to: Jaynetts
Adding: adult standards to: Sam Cooke
Adding: brill building pop to: Crystals
Adding: christmas to: Elvis Presley
Adding: adult standards to: Nat King Cole
Adding: adult standards to: Jackie Wilson
Adding: classic soul to: Marvin Gaye
Adding: classic soul to: Rufus Thomas
Adding: country gospel to: Ned Miller
Adding: adult standards to: Drifters
Adding: adult standards to: Johnny Mathis
Adding: chicago soul to: Jan Bradley
Adding: adult standards to: Bobby Vee
Adding: brill building pop to: The Cookies
Adding: outlaw country to: Johnny Cash
Adding: brill building pop to: Doris Troy
Adding: comic to: Allan Sherman
Adding: adult standards to: Lesley Gore
Adding: unassigned to: Rolf Harris
Adding: brill building pop to: Miracles
Adding: canadian pop to: Dion
Adding: adult standards to: Nat King Cole
Adding: adult standards to: Sam Coo

...

Adding: https://i.scdn.co/image/646fba36b604f30e0c59f9d1043ee0b105ccd5e0 to: Justin Timberlake
Adding: https://i.scdn.co/image/035ad2a01098302eb1d195668fe153c364cb77b1 to: Avril Lavigne
Adding: https://i.scdn.co/image/2403e5037497a7767f74cf832ca642d009cde5a1 to: Keyshia Cole feat. Missy Elliott & Lil Kim
Adding: https://i.scdn.co/image/71e2c7a1026d3a2397ae7ee08797a95d9a16c19f to: Timbaland feat. OneRepublic
Adding: https://i.scdn.co/image/b907bdadb452c37654d528e945c933c36c66555a to: Colbie Caillat
Adding: https://i.scdn.co/image/26da3944201bfadf0095bd6b469bb8e9f53b81e1 to: Ciara
Adding: https://i.scdn.co/image/afc2d276d9c1b398d4f1048c20584f926296b9c0 to: P!nk
Adding: https://i.scdn.co/image/0e874611a108fcd361c92a665a991ba18a852050 to: Kelly Clarkson
Adding: https://i.scdn.co/image/26da3944201bfadf0095bd6b469bb8e9f53b81e1 to: Ciara
Adding: https://i.scdn.co/image/e8a48dd66904570087b66f1196b900554aef78a0 to: J. Holiday
Adding: https://i.scdn.co/image/3911f52e36a9026e218a5dc288e0978704d53

Adding: https://i.scdn.co/image/7403641ccf3013be2e62282c0f00517868fe9ec6 to: The Veronicas
Adding: https://i.scdn.co/image/3cab7a705d1fefebd1f7c8bf03f27587ae697ead to: Nickelback
Adding: https://i.scdn.co/image/6e1c2c7512e8432b4ef2f2fbeb80fdb3c8593ea8 to: Fabolous feat. The-Dream
Adding: https://i.scdn.co/image/449a84503447478d849264a3058d8ea5209750fd to: Boys Like Girls
Adding: https://i.scdn.co/image/27a024d06e8a262508c2e0bfb4c869fc37162b1f to: Asher Roth
Adding: https://i.scdn.co/image/7fa2bde8d2837784a847f8201686b257757a749f to: Britney Spears
Adding: https://i.scdn.co/image/62ce5e006733413abea9a3abfe000acb593dc4ed to: Jason Aldean
Adding: https://i.scdn.co/image/bdaeccb035a8af87b7a70b62217ff5c633ba6c7c to: Taylor Swift
Adding: https://i.scdn.co/image/1fc2f537d678d701d7d143a8fd4f0c2f29fbde22 to: Rihanna
Adding: https://i.scdn.co/image/b6a3f82183adb83b6e47cb22afc25724b241d038 to: Green Day
Adding: https://i.scdn.co/image/ef3d98276e6fd6c8e1d3f3e49f4f3c4fc21d9f8d to: Soulja Boy
Adding

Adding: https://i.scdn.co/image/0b157b64c62c3d783412147dec56b5bb03e7c33c to: Far East Movement feat. Cataracs & Dev
Adding: https://i.scdn.co/image/ce083ade33c047d3f2a72d1cfba3bb2f10e7ecea to: David Guetta feat. Usher
Adding: https://i.scdn.co/image/bdaeccb035a8af87b7a70b62217ff5c633ba6c7c to: Taylor Swift
Adding: https://i.scdn.co/image/fcdc433e8ccf8d46d58ac70db322feb9b3328731 to: Katy Perry
Adding: https://i.scdn.co/image/d3b001ca2d0fd407e4c2a926c767c255dc11633d to: Eli Young Band
Adding: https://i.scdn.co/image/1fc2f537d678d701d7d143a8fd4f0c2f29fbde22 to: Rihanna
Adding: https://i.scdn.co/image/469b6a74f5ddca9560e9f5137842e3772c8576c0 to: Selena Gomez & The Scene
Adding: https://i.scdn.co/image/8a4400fa415a88f54b00e9d5a2927ccf0d74dd51 to: Jake Owen
Adding: https://i.scdn.co/image/a6d6047716a8222f95dda01f46b8ac15658d246a to: Zac Brown Band feat. Jimmy Buffett
Adding: https://i.scdn.co/image/85b7f6d10ae06d21300c9d95f7ea18b35d1df547 to: Luke Bryan
Adding: https://i.scdn.co/image/0c04e8

Adding: https://i.scdn.co/image/25dd6de55a74237ae84e1b53e083bbd5aadade39 to: Ariana Grande feat. Iggy Azalea
Adding: https://i.scdn.co/image/6318ebb61a9105a86c0add8a0a99b633a0301931 to: Sam Smith
Adding: https://i.scdn.co/image/c82dfff224bfb7fe29e0364a5aacc55da29b465b to: Pitbull feat. Ke$ha
Adding: https://i.scdn.co/image/51e9fa0ec4b5aa86398893360acbf14d9bdca88e to: Bastille
Adding: https://i.scdn.co/image/bdaeccb035a8af87b7a70b62217ff5c633ba6c7c to: Taylor Swift
Adding: https://i.scdn.co/image/273b42c39afc5ea4f860ea1cecd6044ea16293ed to: Nico and Vinz
Adding: https://i.scdn.co/image/d079b0bc26a93585e16c34e46bf3425ad0706ce1 to: DJ Snake and Lil Jon
Adding: https://i.scdn.co/image/60c4daa4721f666c6afaee82a39bd413979a0474 to: Eminem feat. Rihanna
Adding: https://i.scdn.co/image/7954fb92b75d26246fa2fb6927601a76a93b43ef to: A Great Big World and Christina Aguilera
Adding: https://i.scdn.co/image/d25fc756cd04c8b3ea196b7c07c6d057685cc405 to: Lorde
Adding: https://i.scdn.co/image/78771cc16b6

Adding: https://i.scdn.co/image/ccf7c1af6ac4a0aa33306f0617eccceaa8d105ba to: Nicky Jam and Enrique Iglesias
Adding: https://i.scdn.co/image/d48ffaffc20a3c76f79d81b7b940df9a380b5e7c to: Ne-Yo feat. Juicy J
Adding: https://i.scdn.co/image/5bb443424a1ad71603c43d67f5af1a04da6bb3c8 to: One Direction
Adding: https://i.scdn.co/image/012ecd119617ac24ab56620ace4b81735b172686 to: Drake
Adding: https://i.scdn.co/image/7b4c05bee8c822c006b372e26ef1b08acc3af45a to: Calvin Harris and Disciples
Adding: https://i.scdn.co/image/5c3cf2ee3494e2da71dcf26303202ec491b26213 to: Justin Bieber
Adding: https://i.scdn.co/image/5c3cf2ee3494e2da71dcf26303202ec491b26213 to: Justin Bieber
Adding: https://i.scdn.co/image/012ecd119617ac24ab56620ace4b81735b172686 to: Drake feat. WizKid and Kyla
Adding: https://i.scdn.co/image/1fc2f537d678d701d7d143a8fd4f0c2f29fbde22 to: Rihanna feat. Drake
Adding: https://i.scdn.co/image/f8e28a81feb9baa0fbc3d69e12587755745a122d to: twenty one pilots
Adding: https://i.scdn.co/image/e285c

In [159]:
df.head()

,Rank,Artist,Song,Year,Artist_ID,Genre,Followers,Img
0,1,Glenn Miller,Chattanooga Choo Choo,1941,2aAHdB5HweT3mFcRzm0swc,adult standards,192386,https://i.scdn.co/image/85cb7d6c2533543abe4043...
1,2,Sammy Kaye,Daddy,1941,54LBMS0P9mMkhfxsp73k1D,unassigned,1770,https://i.scdn.co/image/5dd5b72e62c9e9d159205e...
2,3,Artie Shaw,Stardust,1941,5wam12nGWDBIrLDV78TNSF,adult standards,33225,https://i.scdn.co/image/efda9d6617dfeb8955ef12...
3,4,The Andrews Sisters,Boogie Woogie Bugle Boy,1941,2NCGI6dLTxLdI9XHdv7QfM,adult standards,69409,https://i.scdn.co/image/53ddd415ed44e1dd1f33a6...
4,5,Jimmy Dorsey,Amapola (Pretty Little Poppy),1941,3xPpQCRiTBxgmL4PD8J5ek,big band,14751,https://i.scdn.co/image/c53c748c561a114948219d...


In [200]:
print(len(all_ids_info))

genres_sum = df['Genre'].nunique()
print(genres_sum)
genres_types = df['Genre'].unique()
print(genres_types)
genres_groupby_counts = df.groupby('Genre').nunique()
print(genres_counts)

year_genres_df = df.groupby('Year').nunique()
print(year_genres_df)
genre_value_counts = df.Genre.value_counts()
genres_per_year = df.Year.value_counts()



2589
289
['adult standards' 'unassigned' 'big band' 'country gospel' 'christmas'
 'soda pop' 'choro' 'comic' 'ballroom' 'velha guarda' 'blues'
 'chicago blues' 'bossa nova' 'british dance band' 'bolero' 'pop'
 'vintage jazz' 'bluegrass' 'cabaret' 'cowboy western' 'bebop'
 'vintage western' 'space age pop' 'jazz blues' 'samba' 'easy listening'
 'delta blues' 'jazz trombone' 'contemporary country'
 'deep adult standards' 'vintage schlager' 'movie tunes' 'folk'
 'show tunes' 'ragtime' 'american modern classical' 'lounge' 'doo-wop'
 'classic soundtrack' 'rock-and-roll' 'jazz organ' 'classic girl group'
 'brill building pop' 'acousmatic' 'rockabilly' 'song poem' 'merseybeat'
 'dance pop' 'outlaw country' 'argentine rock' 'blues-rock'
 'appalachian folk' 'rhythm and blues' 'traditional rockabilly'
 'classic italian pop' 'anthem emo' 'country' 'jump blues' 'swedish jazz'
 'jawaiian' 'bass trap' 'nashville sound' 'dixieland' 'latin christian'
 'electronic trap' 'classic schlager' 'american cho

In [183]:
years_df = df.groupby('Year')


print(years_df.get_group(1999))


<bound method NDFrame.head of       Rank                                           Artist  \
4972     1                                             Cher   
4973     2                                              TLC   
4974     3                                           Monica   
4975     4                                  Whitney Houston   
4976     5                                   Britney Spears   
4977     6                         Sixpence None The Richer   
4978     7                               Christina Aguilera   
4979     8                                        Sugar Ray   
4980     9                                      Deborah Cox   
4981    10                                     Ricky Martin   
4982    11                                              702   
4983    12                                   Jennifer Lopez   
4984    13                                    Goo Goo Dolls   
4985    14                                           Brandy   
4986    15               

In [194]:
genres_1941 = years_df.get_group(1941)
genres_1942 = years_df.get_group(1942)
genres_1943 = years_df.get_group(1943)
genres_1944 = years_df.get_group(1944)
genres_1945 = years_df.get_group(1945)
genres_1946 = years_df.get_group(1946)
genres_1947 = years_df.get_group(1947)
genres_1948 = years_df.get_group(1948)
genres_1949 = years_df.get_group(1949)

forties = pd.concat([genres_1941, genres_1942, genres_1943, genres_1944, genres_1945, genres_1946, genres_1947, genres_1948, genres_1949], ignore_index= True)

genres_1950 = years_df.get_group(1950)
genres_1951 = years_df.get_group(1951)
genres_1952 = years_df.get_group(1952)
genres_1953 = years_df.get_group(1953)
genres_1954 = years_df.get_group(1954)
genres_1955 = years_df.get_group(1955)
genres_1956 = years_df.get_group(1956)
genres_1957 = years_df.get_group(1957)
genres_1958 = years_df.get_group(1958)
genres_1959 = years_df.get_group(1959)

fifties = pd.concat([genres_1951, genres_1952, genres_1953, genres_1954, genres_1955, genres_1956, genres_1957, genres_1958, genres_1959], ignore_index= True)

                       
genres_1960 = years_df.get_group(1960)
genres_1961 = years_df.get_group(1961)
genres_1962 = years_df.get_group(1962)
genres_1963 = years_df.get_group(1963)
genres_1964 = years_df.get_group(1964)
genres_1965 = years_df.get_group(1965)
genres_1966 = years_df.get_group(1966)
genres_1967 = years_df.get_group(1967)
genres_1968 = years_df.get_group(1968)
genres_1969 = years_df.get_group(1969)

sixties = pd.concat([genres_1960, genres_1961, genres_1962, genres_1963, genres_1964, genres_1965, genres_1966, genres_1967, genres_1968, genres_1969], ignore_index= True)


genres_1970 = years_df.get_group(1970)
genres_1971 = years_df.get_group(1971)
genres_1972 = years_df.get_group(1972)
genres_1973 = years_df.get_group(1973)
genres_1974 = years_df.get_group(1974)
genres_1975 = years_df.get_group(1975)
genres_1976 = years_df.get_group(1976)
genres_1977 = years_df.get_group(1977)
genres_1978 = years_df.get_group(1978)
genres_1979 = years_df.get_group(1979)

seventies = pd.concat([genres_1970, genres_1971, genres_1972, genres_1973, genres_1974, genres_1975, genres_1976, genres_1977, genres_1978, genres_1979], ignore_index= True)
                       
                       
genres_1980 = years_df.get_group(1980)
genres_1981 = years_df.get_group(1981)
genres_1982 = years_df.get_group(1982)
genres_1983 = years_df.get_group(1983)
genres_1984 = years_df.get_group(1984)
genres_1985 = years_df.get_group(1985)
genres_1986 = years_df.get_group(1986)
genres_1987 = years_df.get_group(1987)
genres_1988 = years_df.get_group(1988)
genres_1989 = years_df.get_group(1989)

eighties = pd.concat([genres_1980, genres_1981, genres_1982, genres_1983, genres_1984, genres_1985, genres_1986, genres_1987, genres_1988, genres_1989], ignore_index= True)
                       
                       
genres_1990 = years_df.get_group(1990)
genres_1991 = years_df.get_group(1991)
genres_1992 = years_df.get_group(1992)
genres_1993 = years_df.get_group(1993)
genres_1994 = years_df.get_group(1994)
genres_1995 = years_df.get_group(1995)
genres_1996 = years_df.get_group(1996)
genres_1997 = years_df.get_group(1997)
genres_1998 = years_df.get_group(1998)
genres_1999 = years_df.get_group(1999)

nineties = pd.concat([genres_1990, genres_1991, genres_1992, genres_1993, genres_1994, genres_1995, genres_1996, genres_1997, genres_1998, genres_1999], ignore_index= True)
                       
                       
genres_2000 = years_df.get_group(2000)
genres_2001 = years_df.get_group(2001)
genres_2002 = years_df.get_group(2002)
genres_2003 = years_df.get_group(2003)
genres_2004 = years_df.get_group(2004)
genres_2005 = years_df.get_group(2005)
genres_2006 = years_df.get_group(2006)
genres_2007 = years_df.get_group(2007)
genres_2008 = years_df.get_group(2008)
genres_2009 = years_df.get_group(2009)

aughts = pd.concat([genres_2000, genres_2001, genres_2002, genres_2003, genres_2004, genres_2005, genres_2006, genres_2007, genres_2008, genres_2009], ignore_index= True)
                       
                       
genres_2010 = years_df.get_group(2010)
genres_2011 = years_df.get_group(2011)
genres_2012 = years_df.get_group(2012)
genres_2013 = years_df.get_group(2013)
genres_2014 = years_df.get_group(2014)
genres_2015 = years_df.get_group(2015)
genres_2016 = years_df.get_group(2016)

tens = pd.concat([genres_2010, genres_2011, genres_2012, genres_2013, genres_2014, genres_2015, genres_2016], ignore_index= True)


In [198]:
genres_1941.to_csv('genres_1941.csv')
genres_1942.to_csv('genres_1942.csv')
genres_1943.to_csv('genres_1943.csv')
genres_1944.to_csv('genres_1944.csv')
genres_1945.to_csv('genres_1945.csv')
genres_1946.to_csv('genres_1946.csv')
genres_1947.to_csv('genres_1947.csv')
genres_1948.to_csv('genres_1948.csv')
genres_1949.to_csv('genres_1949.csv')

forties.to_csv('forties.csv') 

genres_1950.to_csv('genres_1950.csv')
genres_1951.to_csv('genres_1951.csv')
genres_1952.to_csv('genres_1952.csv')
genres_1953.to_csv('genres_1953.csv')
genres_1954.to_csv('genres_1954.csv')
genres_1955.to_csv('genres_1955.csv')
genres_1956.to_csv('genres_1956.csv')
genres_1957.to_csv('genres_1957.csv')
genres_1958.to_csv('genres_1958.csv')
genres_1959.to_csv('genres_1959.csv')

fifties.to_csv('fifties.csv')


genres_1961.to_csv('genres_1961.csv')
genres_1962.to_csv('genres_1962.csv')
genres_1963.to_csv('genres_1963.csv')
genres_1964.to_csv('genres_1964.csv')
genres_1965.to_csv('genres_1965.csv')
genres_1966.to_csv('genres_1966.csv')
genres_1967.to_csv('genres_1967.csv')
genres_1968.to_csv('genres_1968.csv')
genres_1969.to_csv('genres_1969.csv')

sixties.to_csv('sixties.csv')

genres_1970.to_csv('genres_1970.csv')
genres_1971.to_csv('genres_1971.csv')
genres_1972.to_csv('genres_1972.csv')
genres_1973.to_csv('genres_1973.csv')
genres_1974.to_csv('genres_1974.csv')
genres_1975.to_csv('genres_1975.csv')
genres_1976.to_csv('genres_1976.csv')
genres_1977.to_csv('genres_1977.csv')
genres_1978.to_csv('genres_1978.csv')
genres_1979.to_csv('genres_1979.csv')

seventies.to_csv('seventies.csv')

genres_1980.to_csv('genres_1980.csv')
genres_1981.to_csv('genres_1981.csv')
genres_1982.to_csv('genres_1982.csv')
genres_1983.to_csv('genres_1983.csv')
genres_1984.to_csv('genres_1984.csv')
genres_1985.to_csv('genres_1985.csv')
genres_1986.to_csv('genres_1986.csv')
genres_1987.to_csv('genres_1987.csv')
genres_1988.to_csv('genres_1988.csv')
genres_1989.to_csv('genres_1989.csv')

eighties.to_csv('eighties.csv')

genres_1990.to_csv('genres_1990.csv')
genres_1991.to_csv('genres_1991.csv')
genres_1992.to_csv('genres_1992.csv')
genres_1993.to_csv('genres_1993.csv')
genres_1994.to_csv('genres_1994.csv')
genres_1995.to_csv('genres_1995.csv')
genres_1996.to_csv('genres_1996.csv')
genres_1997.to_csv('genres_1997.csv')
genres_1998.to_csv('genres_1998.csv')
genres_1999.to_csv('genres_1999.csv')

nineties.to_csv('nineties.csv')

genres_2000.to_csv('genres_2000.csv')
genres_2001.to_csv('genres_2001.csv')
genres_2002.to_csv('genres_2002.csv')
genres_2003.to_csv('genres_2003.csv')
genres_2004.to_csv('genres_2004.csv')
genres_2005.to_csv('genres_2005.csv')
genres_2006.to_csv('genres_2006.csv')
genres_2007.to_csv('genres_2007.csv')
genres_2008.to_csv('genres_2008.csv')
genres_2009.to_csv('genres_2009.csv')

aughts.to_csv('aughts.csv')

genres_2010.to_csv('genres_2010.csv')
genres_2011.to_csv('genres_2011.csv')
genres_2012.to_csv('genres_2012.csv')
genres_2013.to_csv('genres_2013.csv')
genres_2014.to_csv('genres_2014.csv')
genres_2015.to_csv('genres_2015.csv')
genres_2016.to_csv('genres_2016.csv')

tens.to_csv('tens.csv')

In [203]:
all_years = pd.concat([forties, fifties, sixties, seventies, eighties, nineties, aughts, tens], ignore_index=True)
all_years.to_csv('all_years.csv')

In [ ]:
# @contextlib.contextmanager
# def get_images(url:str):
#   d = soup(requests.get(url).text, 'html.parser') 
#   yield [[i.find('img')['src'], re.findall('(?<=\.)\w+$', i.find('img')['alt'])[0]] for i in d.find_all('a') if re.findall('/image/\d+', i['href'])]

# n = 3 #end value
# os.system('mkdir artist_images')
# for i in range(n):
#   with get_images(f'https://marco.ccr.buffalo.edu/images?page={i}&score=Clear') as links:
#     print(links)
#     for c, [link, ext] in enumerate(links, 1):
#        with open(f'artist_images/artist_img_{i}{c}.{ext}', 'wb') as f:
#            f.write(requests.get(f'https://marco.ccr.buffalo.edu{link}').content)

In [ ]:
top1 = **arrayhere**[0]
top5 = **arrayhere**[:5]
top10 = **arrayhere**[:10]